In [327]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score

## Análise Inicial

In [328]:
df = pd.read_csv("dataset/kaggle_train.csv")


In [329]:
df.head()

,Id,description,address,property_type,price,size,bedrooms,bathrooms,parking_spaces,contract_type,latitude,longitude
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",NaN,kitnet,270000.0,27.0,1.0,NaN,NaN,venda,-15.771441,-47.894002
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",NaN,casa,520000.0,249.0,4.0,NaN,5.0,venda,-15.799918,-48.060482
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",NaN,apartamento,285000.0,98.0,2.0,NaN,2.0,venda,-15.817467,-48.024206
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",NaN,lote,120000.0,380.0,NaN,NaN,NaN,venda,-15.797515,-47.891887
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",NaN,casa,780000.0,300.0,4.0,NaN,3.0,venda,-15.703169,-47.912267


In [330]:
df.columns

Index(['Id', 'description', 'address', 'property_type', 'price', 'size',
       'bedrooms', 'bathrooms', 'parking_spaces', 'contract_type', 'latitude',
       'longitude'],
      dtype='object')

In [331]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12194 entries, 0 to 12193
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              12194 non-null  int64  
 1   description     12194 non-null  object 
 2   address         0 non-null      float64
 3   property_type   12194 non-null  object 
 4   price           12194 non-null  float64
 5   size            11590 non-null  float64
 6   bedrooms        9632 non-null   float64
 7   bathrooms       0 non-null      float64
 8   parking_spaces  7933 non-null   float64
 9   contract_type   12194 non-null  object 
 10  latitude        12194 non-null  float64
 11  longitude       12194 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 1.1+ MB


In [332]:
df.describe()

,Id,address,price,size,bedrooms,bathrooms,parking_spaces,latitude,longitude
count,12194.000000,0.0,12194.000000,11590.000000,9632.000000,0.0,7933.000000,12194.000000,12194.000000
mean,7627.375431,NaN,472782.505248,444.818550,3.021595,NaN,2.758225,-15.862162,-47.975199
std,4402.557957,NaN,246909.622836,4623.596605,1.324415,NaN,2.507233,0.461063,0.260009
min,2.000000,NaN,100000.000000,1.000000,1.000000,NaN,1.000000,-23.758431,-49.271470
25%,3824.250000,NaN,267000.000000,78.000000,2.000000,NaN,1.000000,-15.880037,-48.070185
50%,7594.500000,NaN,420000.000000,154.500000,3.000000,NaN,2.000000,-15.834478,-48.012380
75%,11458.750000,NaN,660000.000000,280.000000,3.000000,NaN,3.000000,-15.797515,-47.913309
max,15243.000000,NaN,999999.000000,250000.000000,22.000000,NaN,60.000000,-3.767821,-38.663459


In [333]:
df["property_type"].value_counts()

property_type
casa           6323
apartamento    2914
lote           2111
rural           560
kitnet          143
sala            143
Name: count, dtype: int64

In [334]:
df["contract_type"].value_counts()

contract_type
venda    12194
Name: count, dtype: int64

In [335]:
df.select_dtypes(exclude=['object']).corr()["price"].sort_values(ascending=False)

price             1.000000
bedrooms          0.393282
parking_spaces    0.255736
Id                0.080621
longitude         0.056772
latitude          0.051069
size              0.038135
address                NaN
bathrooms              NaN
Name: price, dtype: float64

### Feature engineering

In [336]:
df = df.drop(["bathrooms", "address", "contract_type"], axis=1)
df

,Id,description,property_type,price,size,bedrooms,parking_spaces,latitude,longitude
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",kitnet,270000.0,27.0,1.0,NaN,-15.771441,-47.894002
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",casa,520000.0,249.0,4.0,5.0,-15.799918,-48.060482
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",apartamento,285000.0,98.0,2.0,2.0,-15.817467,-48.024206
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",lote,120000.0,380.0,NaN,NaN,-15.797515,-47.891887
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",casa,780000.0,300.0,4.0,3.0,-15.703169,-47.912267
...,...,...,...,...,...,...,...,...,...
12189,5192,"Rua 11, SAO FRANCISCO, SAO SEBASTIAO",casa,810000.0,120.0,3.0,1.0,-23.758431,-45.415684
12190,13419,"QS 417 Conjunto A, SAMAMBAIA NORTE, SAMAMBAIA",lote,300000.0,200.0,NaN,NaN,-15.882325,-48.118433
12191,5391,"Quadra 32, CHACARAS QUEDAS DO DESCOBERTO, AGUA...",casa,450000.0,200.0,3.0,3.0,-15.768771,-48.271678
12192,861,"QN 304, SAMAMBAIA SUL, SAMAMBAIA",apartamento,269000.0,48.0,2.0,1.0,-15.876854,-48.080946


In [337]:
desc = df['description'].str.split(',', expand=True)
df['RA'] = desc.apply(lambda row: next((str(row[i]).strip() for i in range(3, -1, -1) if pd.notnull(row[i])), None), axis=1)
df.head()

,Id,description,property_type,price,size,bedrooms,parking_spaces,latitude,longitude,RA
0,3796,"SGAN 906 Módulo E, ASA NORTE, BRASILIA",kitnet,270000.0,27.0,1.0,NaN,-15.771441,-47.894002,BRASILIA
1,4830,"QNG 2, TAGUATINGA NORTE, TAGUATINGA",casa,520000.0,249.0,4.0,5.0,-15.799918,-48.060482,TAGUATINGA
2,1521,"Rua 4C Chacará 14, VICENTE PIRES, VICENTE PIRES",apartamento,285000.0,98.0,2.0,2.0,-15.817467,-48.024206,VICENTE PIRES
3,13571,"Residencial Damha I Quadra A1, RESIDENCIAL E C...",lote,120000.0,380.0,NaN,NaN,-15.797515,-47.891887,CIDADE OCIDENTAL
4,4541,"GRANJA DO TORTO, GRANJA DO TORTO, BRASILIA",casa,780000.0,300.0,4.0,3.0,-15.703169,-47.912267,BRASILIA


In [338]:
df = df.join(pd.get_dummies(df["RA"], dtype=int))
df = df.join(pd.get_dummies(df["property_type"], dtype=int))
df = df.drop(columns=["description", "RA", "property_type"])

In [339]:
df.head()

,Id,price,size,bedrooms,parking_spaces,latitude,longitude,AGUAS CLARAS,AGUAS LINDAS DE GOIAS,ALPHAVILLE,...,VALPARAISO DE GOIAS,VARJAO,VICENTE PIRES,VILA ESTRUTURAL,apartamento,casa,kitnet,lote,rural,sala
0,3796,270000.0,27.0,1.0,NaN,-15.771441,-47.894002,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,4830,520000.0,249.0,4.0,5.0,-15.799918,-48.060482,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1521,285000.0,98.0,2.0,2.0,-15.817467,-48.024206,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,13571,120000.0,380.0,NaN,NaN,-15.797515,-47.891887,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4541,780000.0,300.0,4.0,3.0,-15.703169,-47.912267,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [340]:
def init_x_test(df):
    desc = df['description'].str.split(',', expand=True)
    df = df.drop(["bathrooms", "address", "contract_type"], axis=1)
    df['RA'] = desc.apply(lambda row: next((str(row[i]).strip() for i in range(3, -1, -1) if pd.notnull(row[i])), None), axis=1)
    df = df.join(pd.get_dummies(df["RA"], dtype=int))
    df = df.join(pd.get_dummies(df["property_type"], dtype=int))
    df = df.drop(["description", "property_type", "RA"], axis=1)
    return df

# Treinando o Modelo

In [341]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [342]:
X_train = df.loc[:, df.columns != "price"]
Y_train = df["price"]

X_test = pd.read_csv("dataset/kaggle_test.csv")
X_test = init_x_test(X_test)

In [343]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12194 entries, 0 to 12193
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           12194 non-null  int64  
 1   size                         11590 non-null  float64
 2   bedrooms                     9632 non-null   float64
 3   parking_spaces               7933 non-null   float64
 4   latitude                     12194 non-null  float64
 5   longitude                    12194 non-null  float64
 6   AGUAS CLARAS                 12194 non-null  int64  
 7   AGUAS LINDAS DE GOIAS        12194 non-null  int64  
 8   ALPHAVILLE                   12194 non-null  int64  
 9   BRASILIA                     12194 non-null  int64  
 10  BRAZLANDIA                   12194 non-null  int64  
 11  CANDANGOLANDIA               12194 non-null  int64  
 12  CEILANDIA                    12194 non-null  int64  
 13  CIDADE OCIDENTAL

In [344]:
X_test.head()

,Id,size,bedrooms,parking_spaces,latitude,longitude,AGUAS CLARAS,AGUAS LINDAS DE GOIAS,ALPHAVILLE,BRASILIA,...,VALPARAISO DE GOIAS,VARJAO,VICENTE PIRES,VILA ESTRUTURAL,apartamento,casa,kitnet,lote,rural,sala
0,4094,130.0,3.0,NaN,-15.895278,-48.127200,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,10766,230.0,5.0,4.0,-16.023340,-48.053761,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,10376,220.0,4.0,2.0,-15.889439,-47.778253,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3775,60.0,2.0,NaN,-15.755336,-47.880407,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,8780,96.0,3.0,2.0,-15.703843,-47.915602,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# Random Forest

In [345]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

predicts = model.predict(X_train)

In [346]:
scores = cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv=10)
model_score = np.sqrt(-scores)

print(f"Scores {model_score}")
print(f"Mean {model_score.mean()}")
print(f"Standart deviation {model_score.std()}")

Scores [104912.48504744 100469.6490429  103682.89999519 104930.20826376
 100596.32953564  94975.78957698  92937.56292161 105035.35569404
 103454.38206104 110359.71319723]
Mean 102135.43753358352
Standart deviation 4865.289174951528


# Avaliação e Otimização


In [347]:
from sklearn.model_selection import GridSearchCV

In [349]:
param_grid = [
    {'n_estimators': [3, 10, 30, 50], 'max_features': [2, 4, 6, 8]},
    {'n_estimators': [5, 10, 20, 30], 'max_features': ['auto', 'sqrt', 'log2'], 'bootstrap': [False]},
]

model = RandomForestRegressor()


grid_search = GridSearchCV(model, param_grid, cv= 5, scoring="neg_mean_squared_error")

grid_search.fit(X_train, Y_train)

c:\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Python312\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Python312\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_p

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30, 50]},
                         {'bootstrap': [False],
                          'max_features': ['auto', 'sqrt', 'log2'],
                          'n_estimators': [5, 10, 20, 30]}],
             scoring='neg_mean_squared_error')

In [350]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 'log2', 'n_estimators': 30}

In [351]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features='log2', n_estimators=30)

In [352]:
model = grid_search.best_estimator_ 
predictions = model.predict(X_test)
predictions

print(f"Scores {model_score}")
print(f"Mean {model_score.mean()}")
print(f"Standart deviation {model_score.std()}")

Scores [104912.48504744 100469.6490429  103682.89999519 104930.20826376
 100596.32953564  94975.78957698  92937.56292161 105035.35569404
 103454.38206104 110359.71319723]
Mean 102135.43753358352
Standart deviation 4865.289174951528


In [353]:
predictions = pd.DataFrame({'Id': X_test['Id'], 'price': predictions})

predictions.to_csv('dataset/predictions.csv', index=False)

In [354]:
X_test['price'] = predictions['price']


In [355]:
X_test

,Id,size,bedrooms,parking_spaces,latitude,longitude,AGUAS CLARAS,AGUAS LINDAS DE GOIAS,ALPHAVILLE,BRASILIA,...,VARJAO,VICENTE PIRES,VILA ESTRUTURAL,apartamento,casa,kitnet,lote,rural,sala,price
0,4094,130.0,3.0,NaN,-15.895278,-48.127200,0,0,0,0,...,0,0,0,0,1,0,0,0,0,273363.266667
1,10766,230.0,5.0,4.0,-16.023340,-48.053761,0,0,0,0,...,0,0,0,0,1,0,0,0,0,615966.666667
2,10376,220.0,4.0,2.0,-15.889439,-47.778253,0,0,0,0,...,0,0,0,0,1,0,0,0,0,320833.333333
3,3775,60.0,2.0,NaN,-15.755336,-47.880407,0,0,0,1,...,0,0,0,1,0,0,0,0,0,678822.666667
4,8780,96.0,3.0,2.0,-15.703843,-47.915602,0,0,0,0,...,0,0,0,0,1,0,0,0,0,286433.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,231,67.0,3.0,1.0,-15.820552,-47.989192,0,0,0,0,...,0,0,0,1,0,0,0,0,0,460683.333333
3045,11765,387.0,NaN,NaN,-15.815916,-48.064199,0,0,0,0,...,0,0,0,0,0,0,1,0,0,168766.666667
3046,4253,400.0,3.0,5.0,-15.797491,-48.037437,0,0,0,0,...,0,1,0,0,1,0,0,0,0,841833.333333
3047,10597,240.0,6.0,3.0,-15.886853,-48.021037,0,0,0,0,...,0,0,0,0,1,0,0,0,0,594833.333333
